### Import Modules

In [ ]:
# Import required modules
import os
import sys
import re
from Bio import SeqIO
import polars as pl

### Function Definitions

In [ ]:
# Function #1: fasta_to_dataframe
def fasta_to_dataframe(fasta):
	"""
	Converts a FASTA file to a polars DataFrame object. 

	Args:
		fasta (str): Path to the FASTA file.

	Returns:
		pl.DataFrame: A DataFrame with 'header' and 'sequence' columns.
	"""

	# Parse the FASTA and extract headers and sequences
	records = SeqIO.parse(fasta, 'fasta')
	# Initialize empty lists to store headers and sequences
	headers = []
	sequences = []

	# Iterate over the records and append the headers and sequnences to the respective lists
	for record in records:
		headers.append(str(record.id))
		sequences.append(str(record.seq))
	
	# Create a polars data frame from the headers and sequences
	df = pl.DataFrame({'headers': headers, 'sequences': sequences})

	return df

# Function #2: find_subsequence_range
def find_subsequence_range(sequence, pattern = "N+"):
	"""
	Find the start and end positions of a subsequence within a larger sequence.

	Args:
		sequence (str): The sequence to search within.
		pattern (str): The subsequence regular expression pattern to search for (Default: "N+").
	Returns:
		List of tuples: List of start and end positions of the subsequence 
		where N's are found. Each tuple is in the form (start, end), where 
		start is zero indexed for a BED file.
	"""

	# Initialize an empty variable to store the current sequence coordinates
	coordinates = []

	# Find all the matches for the pattern in the sequence
	for match in re.finditer(pattern, sequence):
		start, end = match.span()
		# Adjust the end position to be inclusive like in a BED file format
		coordinates.append((start, end - 1))

	# Return the coordinates
	return coordinates


# Function #3: form_dataframe
def form_dataframe(df, pattern = "N+"):
	"""
	Forms a polars DataFrame object from a FASTA file and performs necessary operation to get into BED format.

	Args:
		df (pl.DataFrame): A DataFrame with 'header' and 'sequence' columns created by the fasta_to_dataframe function.
		pattern (str): The subsequence regular expression pattern to search for, which is passed to the find_subsequence_range function (Default: "N+").
	
	Returns:
		pl.DataFrame: A DataFrame with 'header', 'start', and 'end' columns in BED format, with 'start' and 'end'
		being generated by the find_subsequence_range function.
	"""

	# Take the input DataFrame and perform the necessary operations to get it into BED format
	df = (
		df
		.with_columns(
			pl.col('sequences').map_elements(
				lambda x: find_subsequence_range(x, pattern),
				return_dtype = pl.List(pl.List(pl.Int64))
			)
			.alias('coordinates')
		)
		# Drop the sequences column, as it is no longer needed
		.drop(['sequences'])
		# Explode the coordinates column to create a new row for each coordinate
		.explode('coordinates')
		# Create two new columns for the start and end cooordinates
		.with_columns(
			pl.col('coordinates').list.get(0).alias('start'),
			pl.col('coordinates').list.get(1).alias('end')
		)
		# Drop the coordinates column, as it is no longer needed
		.drop(['coordinates'])
		# Drop any rows with null values
		.drop_nulls()
		# Make sure all values are unique
		.unique(maintain_order=True)
	)

	# Return the DataFrame
	return df

In [13]:
# Test function #2
sequence = 'TTCATCAGTGGAACAAGTTGCCTCCAGAAGTTGTGAATGCTGCAATGCTGGAAGTTTTTAAGAAGAGGTTGGACAACCCTTTGTCTGAAGGGGTGCAGGGTTTCCTGCCTAGGCAGGGGGGTTGGACTAGAAGACCTCCAAGGTCCCTTCCAGCTCTTGTtattctattctattctattctattctattctattctattctattctattctattctattctattctattctattctattctattttctattctattctattttctgttctattttctattcttttctattctattctattttctattctattctattttctgttctattttctattctattctattctattctattctctattctattttctattctattctattttctattctatgctattctaACCGTCACAAATAAAGAACTATTTAGGAGTAGAAATAAATAAACTACAAGAAGAAGAGTATGTTCTaagtaaagaaagaaaaaaagaaaATG'
coordinates = find_subsequence_range(sequence, pattern="[acgtryswkmbdhvn]+")
print(coordinates)

[(160, 404), (471, 493)]


### Read in FASTA

In [14]:
# Set the path to the input FASTA file
fasta_file = '/home/administrator/ExtraSSD2/Kaas/Projects/SquamateAlignments/Reference_Genomes/Sekar_Genomes/Scaffold_Assemblies/Elapidae/Naja/Naja_nigricollis_najNig1/Assembly/najNig2.ragtag.scaffold_naNa.REHEADER.MT.fasta'
os.path.exists(fasta_file)

True

### Convert FASTA file to polars DataFrame

In [15]:
# Convert the fasta file to a polars DataFrame
df = fasta_to_dataframe(fasta_file)

In [16]:
df

headers,sequences
str,str
"""MT""","""GTTGTCATAGCTTACCTATCAAGCATAGCA…"
"""scaffoldma1""","""CAGCTGTTCCAACAATCAGCTGTGCCGGNN…"
"""scaffoldma2""","""CTGGGAGGGGAGGCAGGTATTTCCAGTGAT…"
"""scaffoldma3""","""CCCCTCCCTTCCAGCACTGATAATGTTATC…"
"""scaffoldma4""","""AGACACCAGTCCTCTATTTTAATTACAAAT…"
…,…
"""scaffoldun1024""","""GGTTTTCAGGCTTAAGGTGGGATTGGAACT…"
"""scaffoldun1025""","""CCGAACCTTCCTTAGGACTGAAGCTTCTAT…"
"""scaffoldun1026""","""ATTGTAGCAGATAATTTCATGTACTATGCT…"


### Alter the data frame

In [51]:
df_bed = form_dataframe(df)

In [49]:
df_bed

headers,start,end
str,i64,i64
"""scaffoldma1""",28,127
"""scaffoldma1""",35185,35284
"""scaffoldma1""",127631,127730
"""scaffoldma1""",212420,212519
"""scaffoldma1""",217359,217458
…,…,…
"""scaffoldun830""",4266,4365
"""scaffoldun842""",497,596
"""scaffoldun963""",1109,1208


### Write the DataFrame to a BED file

In [52]:
# Write the DataFrame to a BED file
df_bed.write_csv(
	file = '/home/administrator/ExtraSSD2/Kaas/Projects/SquamateAlignments/Reference_Genomes/Sekar_Genomes/Scaffold_Assemblies/Elapidae/Naja/Naja_nigricollis_najNig1/Assembly/najNig2.masked_regions.bed',
	include_header = False,
	separator = "\t"
)